# Assignment 3: Data Collection

In [18]:
# url
url ='https://books.toscrape.com/catalogue/page-2.html'
# scrape over all pages the following informations : title, price, avalaibility, number of product, emage_link , UPC, Product_type, tax, number of reviews, description
# Clean the data

In [19]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as bs

# scrape the following informations

In [20]:
df = pd.DataFrame()
data = []
for index in range(1, 51):

  url = f'https://books.toscrape.com/catalogue/page-{index}.html'
  res = get(url)
  soup = bs(res.content, 'html.parser')
  containers = soup.find_all('li', 'col-xs-6 col-sm-4 col-md-3 col-lg-3')


  for container in containers:
    container_url = 'https://books.toscrape.com/catalogue/' + container.find('a')['href']
    res_container = get(container_url)
    soup_container = bs(res_container.content, 'html.parser')

    try:
      gen_inf = soup_container.find('p', 'instock availability').text.split()

      #title, price, avalaibility, number of product
      title = soup_container.find('div', 'col-sm-6 product_main').h1.text
      price = soup_container.find('p', 'price_color').text.replace('£','')
      avalaibility = " ".join(gen_inf[:2])
      number_prod = gen_inf[2].replace('(', '')


      prod_table = soup_container.find('table', class_ = 'table table-striped')
      info_line = prod_table.find_all('tr')

      upc_line = info_line[0]
      product_type_line = info_line[1]
      price_excl_tax_line = info_line[2]
      price_incl_tax_line = info_line[3]
      tax_line = info_line[4]
      reviews_line = info_line[-1]


      # description, UPC, Product_type,price_excl_tax, price_incl_tax,  tax, number of reviews,
      description_page = soup_container.find('article', 'product_page')
      description = description_page.find('p', recursive=False).text
      UPC = upc_line.find('td').text
      product_type = product_type_line.find('td').text
      price_excl_tax = price_excl_tax_line.find('td').text.replace('£', '')
      price_incl_tax = price_incl_tax_line.find('td').text.replace('£', '')
      tax = tax_line.find('td').text.replace('£', '')
      number_of_reviews = reviews_line.find('td').text

      # Image link
      img_url = 'https://books.toscrape.com/' + container.find('img')['src']


      #dictionary containing variables and their names
      dic = {
            'title' : title,
            'price' : price,
            'avalaibility' : avalaibility,
            'number_prod' : number_prod,
            'description' : description,
            'UPC': UPC,
            'product_type' : product_type,
            'price_excl_tax' : price_excl_tax,
            'price_incl_tax': price_incl_tax,
            'tax': tax,
            'number_of_reviews' : number_of_reviews,
            'image_link': img_url
             }
      data.append(dic)


    except:
      pass
# All extracted data
data_scrape = pd.DataFrame(data)
df = pd.concat([df, data_scrape], axis = 0).reset_index(drop = True)

In [21]:
df.head()

,title,price,avalaibility,number_prod,description,UPC,product_type,price_excl_tax,price_incl_tax,tax,number_of_reviews,image_link
0,A Light in the Attic,51.77,In stock,22,It's hard to imagine a world without A Light i...,a897fe39b1053632,Books,51.77,51.77,0.00,0,https://books.toscrape.com/../media/cache/2c/d...
1,Tipping the Velvet,53.74,In stock,20,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,Books,53.74,53.74,0.00,0,https://books.toscrape.com/../media/cache/26/0...
2,Soumission,50.10,In stock,20,"Dans une France assez proche de la nôtre, un h...",6957f44c3847a760,Books,50.10,50.10,0.00,0,https://books.toscrape.com/../media/cache/3e/e...
3,Sharp Objects,47.82,In stock,20,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,Books,47.82,47.82,0.00,0,https://books.toscrape.com/../media/cache/32/5...
4,Sapiens: A Brief History of Humankind,54.23,In stock,20,From a renowned historian comes a groundbreaki...,4165285e1663650f,Books,54.23,54.23,0.00,0,https://books.toscrape.com/../media/cache/be/a...


In [22]:
df.shape

(998, 12)

# Data Cleanning

In [23]:
df.columns

Index(['title', 'price', 'avalaibility', 'number_prod', 'description', 'UPC',
       'product_type', 'price_excl_tax', 'price_incl_tax', 'tax',
       'number_of_reviews', 'image_link'],
      dtype='object')

In [24]:
# Checking the duplicates
df.duplicated().sum()

np.int64(0)

In [25]:
#checking the types
df.dtypes

,0
title,object
price,object
avalaibility,object
number_prod,object
description,object
UPC,object
product_type,object
price_excl_tax,object
price_incl_tax,object
tax,object


In [26]:
#change of types
df['Price'] = df['price'].astype(float)
df['number_prod'] = df['number_prod'].astype(int)
df['price_excl_tax'] = df['price_excl_tax'].astype(float)
df['price_incl_tax'] = df['price_incl_tax'].astype(float)
df['tax'] = df['tax'].astype(float)
df['number_of_reviews'] = df['number_of_reviews'].astype(float)

In [27]:
df.dtypes

,0
title,object
price,object
avalaibility,object
number_prod,int64
description,object
UPC,object
product_type,object
price_excl_tax,float64
price_incl_tax,float64
tax,float64


# Data recording

In [28]:
df.to_csv('data_books.csv', index = False)